In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [36]:
name = 'bank'
# df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/defaultCredit/data.csv')
df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/{}/data.csv'.format(name))
# target_clm = 'default.payment.next.month'
target_clm = 'y'
#get numerical columns
dtyp_df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/{}/fold0/data_types.csv'.format(name))
num_types = dtyp_df[dtyp_df.type.isin(['pos','count', 'real'])].index
num_types = pd.DataFrame(columns=list(df.columns[num_types]))
cat_types = dtyp_df[~dtyp_df.type.isin(['pos','count', 'real'])].index
num_categories = dtyp_df[~dtyp_df.type.isin(['pos','count', 'real'])].dim
cat_types = df.columns[cat_types]
cat_types = pd.DataFrame(columns=cat_types)
cat_types.loc[0] = num_categories.values
cat_types = cat_types.drop([target_clm], axis=1)
# factorize the categorical columns
df[cat_types.columns] = df[cat_types.columns].apply(lambda x: pd.factorize(x)[0])
#scale numerical columns
scaler = StandardScaler()
# first we need to detect numerical clms and then scale, this is incorrect
df[num_types.columns] = scaler.fit_transform(df[num_types.columns])
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,1.533034,0,0,0,0,0,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
1,1.628993,1,0,1,1,0,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
2,-0.290186,1,0,1,0,1,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
3,-0.002309,2,0,2,0,0,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
4,1.533034,1,0,1,0,0,1,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0


In [38]:
num_types.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/numerical_clms.csv'.format(name), \
     index=False)
cat_types.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/categorical_clms.csv'.format(name), \
     index=False)

In [40]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=142)
df_train, df_valid = train_test_split(df_train, test_size=0.1, random_state=142)
df_train.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/fold0/train/data.csv'.format(name), index=False)
df_valid.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/fold0/valid/data.csv'.format(name), index=False)
df_test.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/fold0/test/data.csv'.format(name), index=False)

In [103]:
cat_types[len(cat_types.index)] = num_categories
# num_categories = num_categories.to_frame(index=False).T

In [114]:
num_categories = num_categories.to_frame().T

<h1> Preparing Large Synthetic Datasert </h1>
<li> Create a large synthetic dataset </li>
<li> Split dataset into training, test and validation </li>
<li> Split train, test and validation into multiple parquet files </li>


In [ ]:
from sklearn.datasets import make_classification

In [ ]:
t_samples = 2000000
feats = 150
X, y = make_classification(
    # same as the previous section
    n_samples=t_samples, n_features=feats, n_informative=100, n_classes=2, 
    # flip_y - high value to add more noise
    flip_y=0.1, 
    # class_sep - low value to reduce space between classes
    class_sep=0.5
)
df = pd.DataFrame(X, columns=['feature_{}'.format(i) for i in range(feats)])
df['label'] = y
df.to_csv('/media/6TB_Volume/DataRepo/LargeSyntheticData/data.csv')

In [9]:
chk = pd.read_csv('/media/6TB_Volume/DataRepo/LargeSyntheticData/data.csv', chunksize=5, index_col=0)
df = next(chk)

In [10]:
num_features = pd.DataFrame(columns=list(df.columns))
num_features.to_csv('/media/6TB_Volume/DataRepo/LargeSyntheticData/numerical_clms.csv', index=False)

In [2]:
# split the dataset into train, test and valid and split them into parquet files
df = pd.read_csv('/media/6TB_Volume/DataRepo/LargeSyntheticData/data.csv', index_col=0)
df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,label
0,4.130683,-2.951145,1.762155,1.303826,-8.245949,0.017939,-5.619700,0.020783,3.019278,5.651979,...,-9.203256,1.425194,9.642056,0.007920,0.063489,-0.548479,4.510751,0.391284,10.083214,1
1,-2.630385,-0.880085,3.945653,-1.330946,0.431737,11.803790,-0.941407,-0.708298,0.848359,-3.481078,...,-5.514812,1.137814,3.203147,0.042314,1.252926,0.482850,-0.629497,0.769117,5.570997,0
2,-1.456159,-0.753166,5.959081,0.527245,2.519723,-1.658751,2.226344,0.220471,-2.017614,-1.558997,...,-0.388120,-0.926325,0.278336,-0.511964,0.483254,-0.671149,9.340674,0.023664,-0.721331,1
3,2.272884,2.053738,5.556700,-1.149170,3.827446,7.726598,-3.137370,-0.692209,-6.006325,9.492668,...,-1.000583,-0.779431,12.268458,-2.118928,1.123373,1.357938,7.964048,1.649285,-2.904829,0
4,0.441287,11.426193,1.252848,0.443455,1.619790,2.914239,2.175868,0.631953,9.852705,1.761268,...,4.123841,-0.472906,2.321520,-0.132357,-1.706263,-0.324381,-3.995659,0.750821,-5.663853,0


In [3]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=142)
df_train, df_valid = train_test_split(df_train, test_size=0.1, random_state=142)

In [7]:
import os
from tqdm import tqdm
def df_to_parquet(df, target_dir, chunk_size=10000, **parquet_wargs):
    """Writes pandas DataFrame to parquet format with pyarrow.

    Args:
        df: DataFrame
        target_dir: local directory where parquet files are written to
        chunk_size: number of rows stored in one chunk of parquet file. Defaults to 1000000.
    """    
    for i in tqdm(range(0, len(df), chunk_size)):
        slc = df.iloc[i : i + chunk_size]
        chunk = int(i/chunk_size)
        fname = os.path.join(target_dir, f"part_{chunk:04d}.parquet")
        slc.to_parquet(fname, engine="pyarrow", **parquet_wargs)
target_dir = '/media/6TB_Volume/DataRepo/LargeSyntheticData/test'
df_to_parquet(df_test, target_dir)

100%|██████████| 40/40 [00:05<00:00,  7.43it/s]
